## Импорт необходимых библиотек

In [46]:
#!pip install ultralytics kagglehub

import os
import zipfile
import shutil
import kagglehub
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from ultralytics import YOLO
import logging
import shutil
from ultralytics import YOLO
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
import torch


## Импорт данных

In [ ]:
shreyj1729_cctv_fights_dataset_path = kagglehub.dataset_download(
    'shreyj1729/cctv-fights-dataset')
gusevoleg_fightsimages_path = kagglehub.dataset_download(
    'gusevoleg/fightsimages')

print('Data source import complete.')

In [ ]:
!kaggle datasets download -d shreyj1729/cctv-fights-dataset -p /content/

In [ ]:
# Распаковка архива с данными
!unzip /content/cctv-fights-dataset.zip -d /content/cctv-fights-dataset

In [ ]:
# Распаковка архива с данными
!unzip /content/violence_maksad.v3-15k.yolov11.zip -d /content/fightsimages/

In [ ]:
for dirname, _, filenames in os.walk('/content/cctv-fights-dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
for dirname, _, filenames in os.walk('/content/fightsimages'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Моделирование

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

data_path = '/content/fightsimages/full/data.yaml'
test_data_path = '/content/fightsimages/full/test/images'
epochs = 50
batch_size = 16

device = 'cuda' if torch.cuda.is_available() else 'cpu'
try:
    logging.info("Инициализация и запуск обучения модели YOLO...")
    model = YOLO('yolov8n.yaml')
    model.train(
        data=data_path,
        epochs=epochs,
        batch=batch_size,
        imgsz=640,
        project='runs/detect/fight_detection_full',
        name='fight_detection_model_full',
        pretrained=True,
        device=device
    )
    logging.info("Обучение модели завершено.")
except Exception as e:
    logging.error(f"Ошибка во время обучения: {e}")

# Запуск TensorBoard в отдельной ячейке/терминале:
# !tensorboard --logdir=runs/detect/fight_detection_full



## Тестирование

In [ ]:
model = YOLO('runs/detect/fight_detection_full/fight_detection_model_full/weights/best.pt')

test_data_path = '/content/fightsimages/full/test/images'

results = model.predict(
    source=test_data_path,
    save=True,
    save_txt=True,
    save_conf=True,
    conf=0.5,
    iou=0.45,
    project='runs/detect/test_results',
    name='test_run'
)


for result in results:
    print(f"Image: {result.path}")
    print("Predictions:", result.boxes.xywh)
    print("Confidence scores:", result.boxes.conf)
    print("Classes:", result.boxes.cls)


## Выгрузка результатов

In [ ]:
try:
    for result in results:
        logging.info(f"Image: {result.path}")
        logging.info("Predictions (xywh):")
        logging.info(result.boxes.xywh)
        logging.info("Confidence scores:")
        logging.info(result.boxes.conf)
        logging.info("Classes:")
        logging.info(result.boxes.cls)
except Exception as e:
    logging.error(f"Ошибка при выводе результатов: {e}")


try:
    results_folder = 'runs/detect/test_results/test_run'
    shutil.make_archive('test_results', 'zip', results_folder)
    logging.info("Архив с результатами тестирования успешно создан: test_results.zip")
except Exception as e:
    logging.error(f"Ошибка при создании архива: {e}")
